In [14]:
import os
import re
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from pyproj import Proj, Transformer

### Optional: Download and combine data

In [ ]:
def read_snowice_bin(filepath):
    rows, cols = 720, 720
    dtype = np.uint8
    
    with open(filepath, 'rb') as f:
        data = np.fromfile(f, dtype=dtype).reshape((rows, cols))
    
    # Mask missing values (255 = missing)
    data = np.where(data == 255, np.nan, data).astype(np.float32)

    return data


def convert_bin_to_netcdf(bin_files, output_nc_path):
    data_list, times = [], []

    # Regex to extract start date YYYYMMDD from filenames like:
    date_pattern = re.compile(r'\.(\d{8})-\d{8}\.')

    for fname in sorted(bin_files):
        data = read_snowice_bin(fname)
    
        match = date_pattern.search(fname)
        if not match:
            print(f"Skipping file (no date found): {fname}")
            continue
    
        date_str = match.group(1)  # e.g. '19800303'
    
        # Use pandas to parse the date string
        start_date = pd.to_datetime(date_str, format='%Y%m%d')

        data_list.append(data)
        times.append(start_date)

    # Stack into 3D array: (time, y, x)
    data_array = np.ma.stack(data_list)

    data_xr = xr.DataArray(
        data_array,
        coords={'time': times, 'y': np.arange(720), 'x': np.arange(720)},
        dims=['time', 'y', 'x'],
        name='snow_ice_classification'
    )

    # Filter to MAM months before resampling
    data_xr = data_xr.sel(time=data_xr['time.month'].isin([3, 4, 5]))

    # Group by 'month' coordinate and average over 'time'
    monthly_means = data_xr.resample(time='1MS').mean()

    # Add lat/lon coordinates for EASE2 25km grid
    nx, ny = 720, 720
    res = 25000  # 25 km grid spacing in meters

    x0 = - (nx // 2) * res
    y0 = (ny // 2) * res

    x = np.arange(nx) * res + x0
    y = y0 - np.arange(ny) * res
    xg, yg = np.meshgrid(x, y)

    proj_ease = Proj('+proj=laea +lat_0=90 +lon_0=0 +datum=WGS84 +units=m')
    transformer = Transformer.from_proj(proj_ease, 'epsg:4326', always_xy=True)
    lon, lat = transformer.transform(xg, yg)

    monthly_means = monthly_means.assign_coords({
        'latitude': (('y', 'x'), lat),
        'longitude': (('y', 'x'), lon),
    })

    # Convert to Dataset
    ds = monthly_means.to_dataset(name='snow_ice_classification')

    ds.to_netcdf(output_nc_path)
    print(f"Saved monthly mean NetCDF to: {output_nc_path}")


if __name__ == "__main__":
    # Example usage
    data_dir = '/pscratch/sd/s/skygale/dyn-data/raw/nsidc0046_data'
    output_nc = '../data/snow_cover_data.nc'

    bin_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.bin')]
    convert_bin_to_netcdf(bin_files, output_nc)


### Analysis

In [53]:
# Load dataset
ds = xr.open_dataset('../data/snow_cover_data.nc')

classif = ds['snow_ice_classification']
lat = ds['lat']
lon = ds['lon']

ds.time.values[:10]

array(['1980-03-01T00:00:00.000000000', '1980-04-01T00:00:00.000000000',
       '1980-05-01T00:00:00.000000000', '1980-06-01T00:00:00.000000000',
       '1980-07-01T00:00:00.000000000', '1980-08-01T00:00:00.000000000',
       '1980-09-01T00:00:00.000000000', '1980-10-01T00:00:00.000000000',
       '1980-11-01T00:00:00.000000000', '1980-12-01T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [46]:
# Create a snow cover mask
def is_snow_covered(classification_data, snow_code=2):
    """Returns a boolean DataArray: True if snow-covered, else False or NaN."""
    return (classification_data == snow_code).where(classification_data != 255)

snow_mask = is_snow_covered(classif, snow_code=2)


In [17]:
# Compute trend per grid cell over time
def snow_cover_trend(snow_mask, time_coord):
    # Convert time to years
    years = pd.to_datetime(time_coord.values).year
    print(years)
    X = years - years.mean()

    def linreg(y):
        mask = np.isfinite(y)
        if np.sum(mask) < 2:
            return np.nan
        return np.cov(X[mask], y[mask])[0, 1] / np.var(X[mask])

    # Convert snow mask to float (0 or 1), so we can regress
    snow_numeric = snow_mask.astype(float)

    # Compute trend of snow frequency over time at each grid point
    trend = xr.apply_ufunc(
        linreg,
        snow_numeric,
        input_core_dims=[["time"]],
        vectorize=True,
        dask="parallelized" if snow_numeric.chunks else False,
        output_dtypes=[float],
    )

    return trend


trend = snow_cover_trend(snow_mask, classif['time'])


Index([1969, 1969, 1969, 1969, 1969, 1969, 1969, 1969, 1969, 1969,
       ...
       1969, 1969, 1969, 1969, 1969, 1969, 1969, 1969, 1969, 1969],
      dtype='int32', length=565)


AttributeError: 'Index' object has no attribute 'mean'

In [11]:
# Plot the trend
def plot_snow_trend(trend, lat, lon, title="Trend in Snow Cover Frequency"):
    plt.figure(figsize=(12, 7))

    # Plot using pcolormesh with 2D lat/lon
    plt.pcolormesh(lon, lat, trend, cmap="RdBu_r", shading="auto")

    plt.colorbar(label="Trend (Δ snow cover presence per year)")
    plt.title(title)
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.tight_layout()
    plt.show()


plot_snow_trend(trend, lat, lon)
